In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader

directory_path = "/kaggle/input/suprema-biostar2-documentation"
all_docs = []

for file in os.listdir(directory_path):
    if file.endswith(".pdf"):
        file_path = os.path.join(directory_path, file)
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        all_docs.extend(docs)

In [2]:
print(type(all_docs))  # <class 'list'>
print(len(all_docs))  # Number of pages or document chunks loaded from all PDFs combined
print(all_docs[0])  # Shows the first document object
print(all_docs[0].page_content)  # Text content of the first document

<class 'list'>
60
page_content='BioStar 2 
Database Table 
 
 
Version 2.6.0 
 
Video Extension 
 
 
 
 
 
 
 
 
 
 
@2018 by Suprema Inc.' metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-02T10:01:36+00:00', 'author': 'ynlee', 'moddate': '2025-06-02T10:01:36+00:00', 'source': '/kaggle/input/suprema-biostar2-documentation/documentation-2.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}
BioStar 2 
Database Table 
 
 
Version 2.6.0 
 
Video Extension 
 
 
 
 
 
 
 
 
 
 
@2018 by Suprema Inc.


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
hf = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

2025-06-02 10:19:21.517173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748859561.550300     398 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748859561.559407     398 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

texts = [doc.page_content for doc in all_docs]

chunks = []
for text in texts:
    chunks.extend(text_splitter.split_text(text))

print(f"Total chunks created: {len(chunks)}")

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_texts(chunks, embedding_model)

vectorstore.save_local("my_faiss_index")

print("Saved to vector store")

Total chunks created: 291


/tmp/ipykernel_398/3300935356.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Saved to vector store


In [5]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()

In [6]:
from langchain_openai import ChatOpenAI
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
OPENROUTER_API_KEY = user_secrets.get_secret("OPENROUTER_API_KEY")
OPENROUTER_API_URL = user_secrets.get_secret("OPENROUTER_API_URL")


model_id = "meta-llama/llama-4-maverick:free"
llm = ChatOpenAI(
    api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_API_URL, model=model_id
)

In [7]:
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]
response = llm.invoke(messages)
print(response.content)

J'adore la programmation.


In [8]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


while True:
    print("1. Search")
    print("2. Exit")
    choice = int(input("Enter your choice : "))
    match choice:
        case 1:
            query = input("Enter your query : ")
            result = qa_chain.run(query)
            print(result)
        case 2:
            print("Thank you..")
            break

1. Search
2. Exit


Enter your choice :  how to add user?


ValueError: invalid literal for int() with base 10: 'how to add user?'

In [11]:
query2 = "How to add user?"
result = qa_chain.run(query2)
print(result)

To add a user, follow these steps:

1. Press the ESC button and authenticate as an administrator.
2. Go to 


In [ ]:
result